In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np
np.random.seed(7)
from keras import backend as K


In [2]:
img_rows = 28
img_cols = 28

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 12

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 3s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same',
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              3137000   
__________

## 1 . Imbalanced Data Set 에서 Focal Loss 사용하는 것이 좋다.

## 2 . 가중치 주는 방법
```

class_weight = {"buy": 0.75,
                "don't buy": 0.25}

model.fit(X_train, Y_train, epochs=10, batch_size=32, class_weight=class_weight)
```

## 3. Over and under sampling

* https://towardsdatascience.com/handling-imbalanced-datasets-in-deep-learning-f48407a0e758
* https://beckernick.github.io/oversampling-modeling/


In [6]:
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [12]:

model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer='adam') 

In [ ]:
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1, 
                 validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 35s 582us/step - loss: 4.7087 - acc: 0.9427 - val_loss: 1.2573 - val_acc: 0.9847
Epoch 2/12
60000/60000 [==============================] - 34s 570us/step - loss: 1.7277 - acc: 0.9805 - val_loss: 0.8890 - val_acc: 0.9894
Epoch 3/12
60000/60000 [==============================] - 34s 569us/step - loss: 1.3192 - acc: 0.9856 - val_loss: 1.2776 - val_acc: 0.9851
Epoch 4/12
60000/60000 [==============================] - 34s 569us/step - loss: 1.0859 - acc: 0.9877 - val_loss: 0.7460 - val_acc: 0.9918
Epoch 5/12
60000/60000 [==============================] - 34s 566us/step - loss: 0.9322 - acc: 0.9889 - val_loss: 0.7140 - val_acc: 0.9915
Epoch 6/12
60000/60000 [==============================] - 34s 568us/step - loss: 0.7639 - acc: 0.9910 - val_loss: 1.1301 - val_acc: 0.9882
Epoch 7/12
60000/60000 [==============================] - 34s 567us/step - loss: 0.7135 - acc: 0.9917 - val_loss: 0.

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])